In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
#!pip install lingpy
#import lingpy
#from lingpy import ipa2tokens
import re

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device("cpu") #For debugging
print(f"Using device: {device}")
torch.set_default_device(device)

KeyboardInterrupt: 

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
dat = pd.read_csv('data/ielexData.csv')

In [ ]:
dat

In [ ]:
dat[dat['Meaning'] == 'few']

In [ ]:
concepts = [re.sub(':.*', '', x) for x in dat['cc']]
dat['concepts'] = concepts
uniqueconcepts = np.unique(concepts)
cognates = [list(re.sub('^.*:', '', x))[0] for x in dat['cc']]
dat['cognate_char'] = cognates

In [ ]:
source_words = []
target_words = []
label = []
for concept in uniqueconcepts:
  tmp_df = dat[dat['concepts'] == concept]
  for word_idx in range(len(tmp_df)):
    source_word = tmp_df['ASJP'].iloc[word_idx]
    source_class = tmp_df['cognate_char'].iloc[word_idx]
    if len(list(source_word)) < 11:
      for word_idx2 in range(len(tmp_df)):
        if word_idx != word_idx2:
          target_word = tmp_df['ASJP'].iloc[word_idx2]
          target_class = tmp_df['cognate_char'].iloc[word_idx2]
          if len(list(target_word)) < 11:
            if source_class == target_class:
              label.append(1)
            else:
              label.append(0)
            source_words.append(source_word)
            target_words.append(target_word)


In [ ]:
source_words[0]

In [ ]:
charlens = []
unique_characters = []
for i in source_words:
  charlens.append(len(list(i)))
  for j in list(i):
    unique_characters.append(j)
unique_characters = ['PAD'] + np.unique(unique_characters).tolist()

In [ ]:
maxlen = max(charlens)

In [ ]:
source_words_tokens = []
target_words_tokens = []
for j in range(len(source_words)):
  source_tmp = [unique_characters.index(i) for i in list(source_words[j])]
  target_tmp = [unique_characters.index(i) for i in list(target_words[j])]
  source_tmp = source_tmp + [0 for x in range(maxlen-len(source_tmp))]
  target_tmp = target_tmp + [0 for x in range(maxlen-len(target_tmp))]

  source_words_tokens.append(source_tmp)
  target_words_tokens.append(target_tmp)

In [ ]:
source_words_tokens[0], target_words_tokens[0]

In [ ]:
unique_characters[25]

In [ ]:
#indices = np.arange(len(source_words_tokens))
#np.random.shuffle(indices)
#split_index = int(0.9 * len(indices))
#train_indices = indices[:split_index]
#test_indices = indices[split_index:]

In [ ]:
#source_words_tokens_train = source_words_tokens[train_indices]
#source_words_tokens_test = source_words_tokens[test_indices]
#target_words_tokens_train = target_words_tokens[train_indices]
#target_words_tokens_test = target_words_tokens[test_indices]
#label_train = label[train_indices]
#label_test = label[test_indices]

In [ ]:
from sklearn.model_selection import train_test_split
source_train, source_test, target_train, target_test, label_train, label_test = train_test_split(source_words_tokens, target_words_tokens, label, test_size=0.1, shuffle=True, random_state=42)

In [ ]:
source_train_tensor = torch.tensor(source_train, dtype=torch.long, device=device)
source_test_tensor = torch.tensor(source_test, dtype=torch.long, device=device)
target_train_tensor = torch.tensor(target_train, dtype=torch.long, device=device)
target_test_tensor = torch.tensor(target_test, dtype=torch.long, device=device)
label_train_tensor = torch.tensor(label_train, dtype=torch.int, device=device)
label_test_tensor = torch.tensor(label_test, dtype=torch.int, device=device)

In [ ]:
train_dataset = TensorDataset(source_train_tensor, target_train_tensor, label_train_tensor)
test_dataset = TensorDataset(source_test_tensor, target_test_tensor, label_test_tensor)

In [ ]:
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size = batch_size, generator=torch.Generator(device=device.type))
test_loader = DataLoader(test_dataset, batch_size = batch_size, generator=torch.Generator(device=device.type))

Next step: NN that takes in both words, converts them to embeddings and predicts whether they are cognates (yes/no) = (1/0), sigmoid classification

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class SimpleCognateDataset(Dataset):
    def __init__(self, data, unique_characters, maxlen):
        self.data = data
        self.char_to_idx = {char: i+1 for i, char in enumerate(unique_characters)}
        self.maxlen = maxlen

    def encode_word(self, word):
        #This is way faster than the "proper method"
        #Because all the data on the cpu device, so making a tensor would send it to the gpu, then pull it from the gpu, then send it again
        #Maybe if we could make this dataset once and put it on the gpu once, then it would be way faster
        encoded = [self.char_to_idx.get(c, 0) for c in word]
        return encoded + [0] * (self.maxlen - len(encoded))
    
        #encoded = [self.char_to_idx.get(c, 0) for c in word]
        #return nn.functional.pad(torch.tensor(encoded, dtype=torch.long), (0, self.maxlen - len(encoded)), value=0)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        word1, word2, label = self.data[idx]
        return (
            torch.tensor(self.encode_word(word1), dtype=torch.long, device=device),
            torch.tensor(self.encode_word(word2), dtype=torch.long, device=device),
            torch.tensor(label, dtype=torch.float, device=device)
        )

class SimplePairNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=128):
        super(SimplePairNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size + 1, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def encode_word(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        return hidden[-1]

    def forward(self, input1, input2):
        enc1 = self.encode_word(input1)
        enc2 = self.encode_word(input2)
        combined = torch.cat([enc1, enc2], dim=1)
        return self.fc(combined)

In [ ]:
import pandas as pd
from itertools import combinations

df = pd.read_csv("data/ielexData.csv")

df = df[['Meaning', 'Phonological Form', 'cc']].dropna()
df.columns = ['meaning', 'word', 'cognate_class']

ilexPairs = []

for _, group in df.groupby('meaning'):
    entries = group.to_dict('records')
    for w1, w2 in combinations(entries, 2):
        word1 = str(w1['word'])
        word2 = str(w2['word'])
        label = int(w1['cognate_class'] == w2['cognate_class'])
        ilexPairs.append((word1, word2, label))

import csv

# Initialize an empty set to store the languages
languages = set()

# Open and read the CSV file with UTF-8 encoding and error handling
with open('data/ielexData.csv', 'r', encoding='utf-8', errors='replace') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        languages.add(row[1])  # Add the language (second column) to the set

# Print the set of languages
print(languages)

df

In [ ]:
df = pd.read_csv("data/gled.tsv", delimiter="\t")

df.rename(columns={'LANGUAGE_NAME': 'Language'}, inplace=True)

df['Language'] = df['Language'].str.replace('Modern Greek', 'Greek', regex=False)

df = df[df['Language'].isin(languages)]

#df.dropna(subset=["COGSET"], inplace=True) All of them have a cogset

df

In [ ]:
df = df[['CONCEPT', 'FORM', 'COGSET']].dropna()
df.columns = ['meaning', 'word', 'cognate_class']

pairs = []

for _, group in df.groupby('meaning'):
    entries = group.to_dict('records')
    for w1, w2 in combinations(entries, 2):
        word1 = str(w1['word'])
        word2 = str(w2['word'])
        if(word1 == word2):
            continue
        label = int(w1['cognate_class'] == w2['cognate_class'])
        pairs.append((word1, word2, label))

In [ ]:
len(pairs)

In [ ]:
pairs += ilexPairs

In [ ]:
pairs

In [ ]:
pairs_df = pd.DataFrame(pairs, columns=['Word1', 'Word2', 'Label'])
pairs_df

In [ ]:
from sklearn.model_selection import LeaveOneOut, KFold, StratifiedKFold
import numpy as np
from tqdm import tqdm
import time

# Prepare data
all_words = [w for pair in pairs for w in pair[:2]]
all_answers = [w for pair in pairs for w in pair[2:]]
unique_characters = sorted(set("".join(all_words)))
maxlen = max(len(w) for w in all_words)

import matplotlib.pyplot as plt
def show_box_plot(data, title="Cross Validation Accuracies"):
	f = plt.figure()
	f.set_figwidth(4)
	f.set_figheight(1)

	# Create a box and whiskers plot
	plt.boxplot(data, vert=False)

	plt.scatter(data, [1]*len(data), color='blue', alpha=0.3, s=20)

	# Add title and labels
	plt.title(title)
	plt.ylabel('Values')

	# Show the plot
	plt.show()

def show_confusion_matrix(cm, title="Confusion Matrix"):
	fig, ax = plt.subplots()
	cax = ax.matshow(cm, cmap=plt.cm.Blues)

	for (i, j), z in np.ndenumerate(cm):
		ax.text(j, i, '{:0.1f}'.format(z), ha='center', va='center',
				bbox=dict(boxstyle='round', facecolor='white', edgecolor='0.3'))
	
	ax.set_xticks([0, 1])
	ax.set_yticks([0, 1])   
	ax.set_xticklabels(['Predicted Negative', 'Predicted Positive'])
	ax.set_yticklabels(['True Negative', 'True Positive'])
	plt.xlabel('Predicted')
	plt.ylabel('True')
	plt.title(title)

def cv_test_model(model_maker, has_history=False):
	cv = StratifiedKFold(n_splits = 10)
	epochs = 1500

	# Store results for each fold
	# All of these are [epoch_number, fold, (confusion_matrix, accuracy)]
	
	history = {
		"train": [
			[tuple((None, None)) for _ in range(cv.get_n_splits())]
			for _ in range(epochs)
		],
		"test": [
			[tuple((None, None)) for _ in range(cv.get_n_splits())]
			for _ in range(epochs)
		]
	}

	for fold, (train_idx, test_idx) in enumerate(tqdm(cv.split(pairs_df.iloc[:, :2], pairs_df.iloc[:, 2:]), total=cv.get_n_splits(), desc="CV Progress")):
		# Split data
		train_data = [pairs[i] for i in train_idx]
		test_data = [pairs[i] for i in test_idx]
		
		# Create model for this fold
		model = model_maker()
		criterion = nn.BCELoss()
		optimizer = optim.Adam(model.parameters(), lr=0.001)
		
		# Create datasets and dataloaders
		train_dataset = SimpleCognateDataset(train_data, unique_characters, maxlen)
		test_dataset = SimpleCognateDataset(test_data, unique_characters, maxlen)
		
		train_loader = DataLoader(train_dataset, batch_size=3200, shuffle=True, 
								generator=torch.Generator(device=device.type))
		test_loader = DataLoader(test_dataset, batch_size=3200, 
								generator=torch.Generator(device=device.type))
		
		
		def test_against_loader(model, loader):
			predictions = []
			true_labels = []

			with torch.no_grad():
				for word1_batch, word2_batch, label_batch in loader:
					output_batch = model(word1_batch, word2_batch).squeeze()
					predicted_batch = (output_batch > 0.5).int()

					# Store results for the entire batch
					predictions.extend(predicted_batch.tolist())
					true_labels.extend(label_batch.int().tolist())

			# Calculate accuracy
			accuracy = sum([pred == true for pred, true in zip(predictions, true_labels)]) / len(true_labels)

			# Additional metrics
			from sklearn.metrics import confusion_matrix
			
			# Calculate confusion matrix
			conf_matrix = confusion_matrix(true_labels, predictions)

			return conf_matrix, accuracy

		# Train model
		model.train()
		for epoch in range(epochs):
			total_loss = 0
			for word1, word2, label in train_loader:
				optimizer.zero_grad()
				output = model(word1, word2).squeeze()
				#print(output.get_device(), label.get_device())
				loss = criterion(output, label)
				loss.backward()
				optimizer.step()
				total_loss += loss.item()
			#print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

			if has_history:
				model.eval()
				history["train"][epoch][fold] = test_against_loader(model, train_loader)
				history["test"][epoch][fold] = test_against_loader(model, test_loader)

				#Early cutoff if overfitting
				if history["train"][epoch][fold][1] >= .99:
					history["train"] = history["train"][0:epoch]
					history["test"] = history["test"][0:epoch]
					return history, model
				
				model.train()

		if not has_history:
			history["test"][-1][fold] = test_against_loader(model, test_loader)

	return history, model


In [ ]:
from transformer_stuff import TransformerCognateModel, UnbatchedWrapper

def simple_nn_model_maker():
    # Model parameters
    embedding_dim = 64
    hidden_dim = 64
    vocab_size = len(unique_characters)

    model = TransformerCognateModel(vocab_size, embedding_dim, hidden_dim)
    return model
    #return UnbatchedWrapper(model)

history, model = cv_test_model(simple_nn_model_maker, has_history=True)

In [ ]:
# Calculate overall statistics
last_epoch_test_data = history["test"][-1] #[fold, (pred, labels, accuracies, accuracy)]

all_confusion = np.sum(np.array([fold_data[0] for fold_data in last_epoch_test_data]), axis=0)
all_accuracies = [fold_data[1] for fold_data in last_epoch_test_data]

overall_accuracy = np.mean(all_accuracies)
correct_predictions = all_confusion[1][1] + all_confusion[0][0]
total_predictions = np.sum(all_confusion)

print(f"\nCross-Validation Results:")
print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")
print(f"Correct Predictions: {correct_predictions}/{total_predictions}")

fold_accuracies = [fold_data[1] for fold_data in history["train"][-1]]
show_box_plot(fold_accuracies)

show_confusion_matrix(all_confusion)

# Initialize lists to store accuracies for each fold
train_fold_accuracies = np.zeros((len(history["train"][-1]), len(history["train"])))
test_fold_accuracies = np.zeros((len(history["train"][-1]), len(history["train"])))

# Extract training and testing accuracies for each fold across all epochs
for epoch_index, epoch_data in enumerate(history["train"]):
    for fold_index, fold_data in enumerate(epoch_data):
        train_fold_accuracies[fold_index][epoch_index] = fold_data[1]

for epoch_index, epoch_data in enumerate(history["test"]):
    for fold_index, fold_data in enumerate(epoch_data):
        test_fold_accuracies[fold_index][epoch_index] = fold_data[1]

# Plot training and testing accuracies for each fold on one graph
plt.figure(figsize=(10, 5))
for fold_index, accuracies in enumerate(train_fold_accuracies):
    plt.plot(accuracies, color='orange', alpha=.4)
for fold_index, accuracies in enumerate(test_fold_accuracies):
    plt.plot(accuracies, color='blue', alpha=.4)

# Add custom legend
leg = plt.legend(['Training', 'Testing'], loc='upper left')
leg.legend_handles[0].set_color('orange')
leg.legend_handles[1].set_color('blue')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.xticks(np.arange(1, len(history["train"]), step=1))
plt.title('Training and Testing Accuracies Over Epochs')
plt.show()

In [ ]:
len(train_fold_accuracies[1])

In [ ]:
test_fold_accuracies[-1]

In [ ]:
history

In [ ]:
def simple_nn_model_maker():
    # Model parameters
    embedding_dim = 64
    hidden_dim = 128
    vocab_size = len(unique_characters)

    return SimplePairNN(vocab_size, embedding_dim, hidden_dim)

history, model = cv_test_model(simple_nn_model_maker, has_history=True)

In [ ]:
final_model = SimplePairNN(vocab_size, embedding_dim, hidden_dim)
final_criterion = nn.BCELoss()
final_optimizer = optim.Adam(final_model.parameters(), lr=0.1)

final_dataset = SimpleCognateDataset(pairs, unique_characters, maxlen)
final_loader = DataLoader(final_dataset, batch_size=32, shuffle=True, 
                         generator=torch.Generator(device='cuda'))

for epoch in tqdm(range(5), desc="Training final model"):
    final_model.train()
    total_loss = 0
    for word1, word2, label in final_loader:
        final_optimizer.zero_grad()
        output = final_model(word1, word2).squeeze()
        loss = final_criterion(output, label)
        loss.backward()
        final_optimizer.step()
        total_loss += loss.item()
    print(f"Final model - Epoch {epoch+1}/5, Loss: {total_loss / len(final_loader):.4f}")

final_model.eval()
final_predictions = []
final_true_labels = []

with torch.no_grad():
    for word1, word2, label in tqdm(final_loader, desc="Evaluating final model"):
        output = final_model(word1, word2).squeeze()
        predicted = (output > 0.5).int()
        
        # Handle both single samples and batches
        if predicted.dim() == 0:
            final_predictions.append(predicted.item())
            final_true_labels.append(label.int().item())
        else:
            final_predictions.extend(predicted.cpu().numpy())
            final_true_labels.extend(label.int().cpu().numpy())

# Calculate final model metrics
final_accuracy = np.mean(np.array(final_predictions) == np.array(final_true_labels))
final_precision = precision_score(final_true_labels, final_predictions, zero_division=0)
final_recall = recall_score(final_true_labels, final_predictions, zero_division=0)
final_f1 = f1_score(final_true_labels, final_predictions, zero_division=0)
final_cm = confusion_matrix(final_true_labels, final_predictions)

print(f"\nFinal Model Performance (on all data):")
print(f"Accuracy: {final_accuracy * 100:.2f}%")
print(f"Precision: {final_precision:.3f}")
print(f"Recall: {final_recall:.3f}")
print(f"F1-Score: {final_f1:.3f}")
print(f"Confusion Matrix:")
print(final_cm)

In [ ]:
def predict_pair(word1, word2):
    def encode(w):
        char_to_idx = {char: i+1 for i, char in enumerate(unique_characters)}
        idxs = [char_to_idx.get(c, 0) for c in w]
        return idxs + [0] * (maxlen - len(idxs))

    w1 = torch.tensor([encode(word1)], dtype=torch.long, device=device)
    w2 = torch.tensor([encode(word2)], dtype=torch.long, device=device)
    final_model.eval()
    with torch.no_grad():
        out = final_model(w1, w2)
    return "Yes" if out.item() > 0.5 else "No"

https://github.com/pytorch/examples/tree/main/word_language_model